# Семинар 1. Предобработка текстовых данных

В начале любой работы с текстом, как правило, требуется выполнить одни и теже действия: удалить все лишнее, разбить на предложения, токенизировать, нормализовать. На этом занятии мы разберем каждый из этих этапов. 

In [15]:
# сразу импортируем все нужные библиотеки
# подробнее о каждой из них я расскажу по ходу 
import string
from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import re, os, json
mystem = Mystem()
morph = MorphAnalyzer()

## Удаление лишнего

Часто в данных, с которыми нам нужно работать помимо текста присутствует ещё какая-то лишняя информация: тэги, ссылки, код, разметка. Она, конечно, не всегда лишняя, но обычно от неё лучше избавиться.

Возьмем в качестве примера статью с Хабрахабра. Она скачана автоматически и в там остались некоторые тэги.

In [16]:
text = '''
Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!<br><br><img src="http://img172.imageshack.us/img172/8434/18274658kc4.png" alt="Сумасшедшее яйцо"><br><br><blockquote><h3>Что это такое?</h3><br>
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.<br>
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).<br><br><img src="https://habrastorage.org/getpro/habr/post_images/7c7/298/33c/7c729833cd942cc493e68833e3e0f12d.jpg" alt="Тепловое отслеживание популярности"><br></blockquote><br><br><a name="habracut"></a><br><br><blockquote><h3>Для кого это?</h3><br>
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн дает о себе знать — сервис массово популяризировался пол года-год назад. Относительно недорогой, но уж точно не из дешевых — сервис предполагает собой 4 платных линии и одну бесплатную. Дабы написать этот обзор я не поленился заплатить 19 долларов (в месяц) выбрав средний вариант — для нескольких проектов с включенными дополнительными функциями.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0b0/0a0/b16/0b00a0b16b1eda28e35f39487dcd2545.jpg" alt="Отслеживание ссылок"></blockquote><br><blockquote><img src="https://habrastorage.org/getpro/habr/post_images/0b5/433/892/0b54338921665ffb5a90930147296f5b.jpg" alt="Список по популярности"><br><br>
Да-да, сервис не бесплатен. Точнее бесплатная возможность потестировать есть, но она немного обрезана (можно отслеживать только 5000 посетителей и всего 4 страницы на сайте (внимание — 4 страницы, а не сайтов), т.е. вполне хватает для того, чтобы понять полезность сервиса).</blockquote><br><blockquote><h3>Зачем это?</h3><br>
С помощью этого сервиса можно тасовать блоки на сайте, которые полезны пользователям больше всего. Больше не нужно спорить создателям — какой блок где расположить. В этом им поможет <a href="http://ttp://www.crazyegg.com" title="Сумасшедшие яйца">Сумасшедшее яйцо</a>.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/38c/af8/e75/38caf8e753782a01dc6419d3902edd57.jpg" alt="Добавление проекта"></blockquote><br><blockquote>Также этот сервис поможет вам понять — в какой зоне сайта лучше всего располагать рекламу, когда в вашем сервисе речь зайдет о монетизации. Ведь альтруизм это хорошо, а деньги на содержание сервиса нужны, и не лишним будет вычислить зоны где реклама будет приносить наибольшую отдачу, и наименьшее раздражение у пользователей.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0dd/3fb/b34/0dd3fbb34709e77f3fbcd6523c7eac77.jpg" alt="еще полезности"></blockquote><br><blockquote><h3>Как это работает?</h3><br>
Никаких километровых скриптов вставлять не нужно, достаточно вставиь 2 строчки яваскрипта, и сервис начнет отслеживание. Насколько я понял — исполнительный скрипт работает на сервере <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>, поэтому ваш сайт от этого в производительности не потеряет ни секунды, а полезность довольно таки большая.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/a90/e4e/e67/a90e4ee6760978463e6306a2f5982e24.jpg" alt="управление популярностью"></blockquote><br><blockquote><h3>Сколько это стоит?</h3><br>
Как я уже упоминал — сервис далеко не бесплатен, хоть и имеет тестовую-бесплатную версию. Расскажу подробнее о тарификации.</blockquote><br><blockquote>1. Бесплатная тестовая версия. Включает в себя возможность отслеживания 5 000 посещений, 4 страницы на сайте.<br><br>
2. Базовая версия. В этой версии можно отследить 10 000 посещений, и 10 страниц, что вполне достаточно для среднего корпоративного сайта. Стоит базовый комплект — 9 долларов в месяц. В этот комплект включены все дополнительные функции.<br><br>
3. Версия «Стандарт». В неё входит возможность отслеживания 25 000 посещений на 20 страницах. Вполне подходит для тестирования нового стартапа. Стоит она 19 долларов в месяц, именно её я купил для тестирования сервиса, и написания этого обзора.<br><br>
4. Версия «Плюс». Отличается от предыдущей возможностью отслеживания 100 000 посещений, 50 страниц. Очень хороший тариф для крупных сервисов. Стоит 49 долларов в месяц. Довольно большие деньги за сервис, но они обычно с лихвой окупаются.<br><br>
5. Версия «Про». Стоит почти 100 долларов, имеет возможность отследить 250 000 посещений на ста страницах. Тариф подходит для монстров с большой посещаемостью и большим количеством страниц.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/f73/fb2/62d/f73fb262da4618a8dde67690cfd191ea.jpg" alt="Отслеживание статистики"></blockquote><br><blockquote><h3>Есть и аналоги</h3><br>
Я не поленился, и собрал еще пару ссылок с аналогами, которые предлагают такие же услуги, но немного дешевле.</blockquote><br><blockquote> 1. <a href="https://www.google.com/analytics/home/?hl=en" title="Шикарный сбор и анализ статистики">Google Analytics</a> — бесплатный сервис для сбора и анализа статистики, вывод статистики в наиболее наглядной форме, и без разнообразных рейтингов, счетчиков. Очень подробная и полезная вещь. Рекомендую, т.к. сам пользуюсь им для этого блога.<br><br>
2. <a href="http://www.mapsurface.com/" title="Сервис для отслеживания популярности блоков на сайте">MapSurface</a> — сам еще не использовал (т.к. предпочел CrazyEgg), но врядли будучи плохим сервис собрал бы множество положительных отзывов. К сожалению он сейчас находится в статусе закрытой беты.</blockquote><br><blockquote><h3>Вывод</h3><br>
Использовать можно, и нужно. Вот только тарифы довольно больно кусаются, но обычно эти деньги потом с лихвой отбиваются на повышении конвертации посетителей в деньги. Использовать сервис нужно для тестирования рекламных мест и удобства отдельных страниц, что помогает опять же повысить конвертацию. В общем и целом — <a href="http://ttp://www.crazyegg.com">полезная вещь</a> для каждого владельца сайтов, а для юзабилиста вообще практически обязательна. К счастью для людей, которые поиздержались деньгами в этом месяце — есть полезные аналоги.<br><br><img src="http://img241.imageshack.us/img241/2346/confettitnml5.jpg" alt="Интересная идея визуализации популярности - конфети"></blockquote><br><br>
Автор: <a href="http://www.birzool.com/" title="Я пишу о юзабилити веб интерфейсов">Ярослав Бирзул</a> (DezmASter).<br>
Источник: <a href="http://www.birzool.com/crazyegg/" title="Сумасшедшие яйца, первоисточник">Блог о юзабилити веб интерфейсов</a>.<br><br>
PS: Всех с прошедшим Новым годом! От всей души желаю вам всего самого-самого лучшего, чего вы желаете только в самых сокровенных мечтах. Удачно вам провести время.
'''

В html все тэги заключаются в фигурные скобки. Мы можем испьзовать это, чтобы легко избавиться от всех тэгов сразу. Напишем регулярное выражение, которое будет захватывать всё, что попадает между символами < и >, и не является '>'.

In [12]:
#re - модуль регулярных выражений в питоне
# функция sub заменяет все, что подходит под шаблон, на указанный текст
def remove_tags_1(text):
    return re.sub(r'<[^>]+>', '', text)

Небольшое отступление: ещё есть модуль regex, который умеет все, что и re, а также много всего другого. Подробнее про него можно почитать тут - https://pypi.org/project/regex/

Проверим как работает наша функция.

In [13]:
print(remove_tags_1(text))


Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!Что это такое?
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).Для кого это?
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн да

Можно заметить, что в некоторых местах удаление тэгов приводит к тому, что между точкой и началом следующего 
предложения нет пробела, а это может помешать правильно токенизировать текст,
поэтому сделаем так, чтобы тэг заменялся пробелом

In [14]:
def remove_tags_2(text):
    return re.sub(r'<[^>]+>', ' ', text)

In [16]:
print(remove_tags_2(text))


Сегодняшняя заметка будет о сервисе отслеживания активности пользователя —  CrazyEgg . Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!       Что это такое?  
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов. 
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).             Для кого это?  
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный инт

Теперь куски текста не слепаются, но появились последовательности из нескольких пробелов, чтобы убрать их добавим ещё одно регулярное выражение и применим его к тексту, из которого уже удалили тэги

In [18]:

def remove_tags_3(text):
    no_tags_text = re.sub(r'<[^>]+>', ' ', text)
    no_space_sequences_text = re.sub('  +', ' ', no_tags_text)
    return no_space_sequences_text

In [19]:
print(remove_tags_3(text))


Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg . Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично! Что это такое? 
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов. 
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые). Для кого это? 
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший диз

Теперь текст более менее чистый.

In [21]:
text = remove_tags_3(text)

## Разбиение на предложения

Разбить текст на предложения тоже можно с помощью регулярных выражений. Однако в этом случае регулярка не такая простая. 

Для разбиения текста по какому-то определенному разделителю подходит функция re.split. Давайте посмотрим, что получится, если в качестве разделителя использовать !?. пробел и заглавную букву.  

In [19]:
re.split('[!?\.] [А-Я]', text)[:10]

['\nСегодняшняя заметка будет о сервисе отслеживания активности пользователя — <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>',
 ' не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо',
 'апоминается',
 'тлично!<br><br><img src="http://img172.imageshack.us/img172/8434/18274658kc4.png" alt="Сумасшедшее яйцо"><br><br><blockquote><h3>Что это такое?</h3><br>\nКак уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.<br>\nСервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остал

Проблема в том, что сам разделитель удаляется тоже, а нам нужно удалить только пробел между знаками препинания и заглавной буквой.

Решается эта проблема с помощью __look ahead__ и __look behind__ (название функционала в регулярных выражениях).  
Синтаксис там такой:  
**(?<=pattern)** положительное look-behind условие  
**(?<!pattern)** отрицательное look-behind условие  
**(?=pattern)** положительное look-ahead условие  
**(?!pattern)** отрицательное look-ahead условие  

Подробно про это написано тут: https://www.regular-expressions.info/lookaround.html  
А совсем подробно про (?...) выражения вообще вот тут - https://www.rexegg.com/regex-disambiguation.html

Look behind и look ahead превращают паттерн в условный, то есть проверяется есть ли он (до или после, соответственно), но его захвата не происходит. 

Обернём наше регулярное выражение и посмотрим, что получается:

In [142]:
re.split('(?<=[\.?!]) +(?=[А-ЯЁ])', text.replace('\n', ' '))[:10]

[' Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg .',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?',
 'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.',
 'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).',
 'Для кого это?',
 'Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуи

В __nltk__ есть уже готовая функция для разбивки на предложения. 

In [144]:
sent_tokenize(text, 'russian')[:10]

['\nСегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg .',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?',
 'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.',
 'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).',
 'Для кого это?',
 'Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — инту

В __gensim__ тоже есть готовая функция

In [203]:
# это ещё и генератор, т.е. сразу подходит для больших корпусов
list(split_sentences(text))[:5]

['Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg .',
 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.',
 'Запоминается?',
 'Отлично!',
 'Что это такое?']

## Токенизация

Мы разбили текст на предложения. Теперь предложения нужно разбить на токены. Под токенами обычно понимаются слова, но это могут быть и какие-то более длинные или короткие куски. 

Самый простой способ токенизации -- стандартный питоновский __str.split__ метод.
    По умолчанию он разбивает текст по последовательностям пробелом 
 (т.е. даже со второй версией remove_tags всё бы хорошо разделилось)

In [1140]:
text.split()[:20]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 '—',
 'CrazyEgg',
 '.',
 'Я',
 'не',
 'знаю',
 'кому',
 'обязан',
 'сервис',
 'таким',
 'говорящим',
 'именем,']

Большая часть слов отделяется, но знаки препинания лепятся к словам.
Можно пройтись по всем словам и убрать из них пунктцацию с методом str.strip.

In [1124]:
#основные знаки преминания хранятся в питоноском модуле string.punctuation
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [148]:
# в этом списке не хватает кавычек-ёлочек, лапок, длинного тире и многоточия
string.punctuation += '«»—…“”'

In [149]:
[word.strip(punct_extended) for word in text.split()][:10]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 '',
 'CrazyEgg']

Так не будут удаляться дефисы и точке в сокращениях, не разделенных пробелом.

In [150]:
'как-нибудь'.strip(punctuation)

'как-нибудь'

In [151]:
'т.е.'.strip(punctuation)

'т.е'

Ещё слова можно извлечь с помощью простого регулярного выражения:

In [154]:
re.findall('\w+', text)[:10]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 'CrazyEgg',
 'Я']

Как-нибудь так разделится, но это все-таки не так страшно.

Ещё есть готовые токенизаторы из nltk. Они не удаляют пунктуацию, а выделяют её отдельным токеном.

Например **wordpunct_tokenizer** разбирает по регулярке - *'\w+|[^\w\s]+'* 

In [1149]:
wordpunct_tokenize(clean_habr_texts[1])[:10]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 '—',
 'CrazyEgg']

Ещё есть **word_tokenize**. Он также построен на регулярках, но они там более сложные (учитывается последовательность некоторых 
символов, символы начала, конца слова и т.д). 

Специально подобранного под русский язык токенизатора там нет, 
но и с английским всё работает достаточно хорошо
сокращения типа т.к собираются в один токен, дефисные слова тоже не разделяются
Многоточия тут тоже не отделяются, но это можно поправить.

In [1132]:
word_tokenize(clean_habr_texts[1])[:10]

['Сегодняшняя',
 'заметка',
 'будет',
 'о',
 'сервисе',
 'отслеживания',
 'активности',
 'пользователя',
 '—',
 'CrazyEgg']

В генсиме тоже есть функция для токенизации

In [206]:
# опять же, это генератор
list(tokenize(text, lowercase=True))[:5]

['сегодняшняя', 'заметка', 'будет', 'о', 'сервисе']

# Нормализация

Полученные токены тоже чаще всего нужно привести к какому-то стандартному виду.

Самое простое и очевидное - привести всё к одному регистру:

In [46]:
'СловО'.lower()
# если не нужно разбивать на предложения, то это можно сделать в самом начале
# text.lower()

'слово'

Для языков со слабым словоизменением этого может быть достаточно. Но для русского с его склонениями и спряжениями лучше использовать стемминг или лемматизацию.

## Стемминг

Стемминг - это нахождение основы слова, т.е. такой части, которая является общей для всех словоформ в парадигме слова. На практике стемминг сводится к отбрасыванию частотных окончаний.

Подробнее про стеммер Портера можно почитать вот тут - <https://medium.com/@eigenein/стеммер-портера-для-русского-языка-d41c38b2d340>

А совсем подробнее вот тут - <http://snowball.tartarus.org/algorithms/russian/stemmer.html>

Готовые стеммеры для разных языков есть в nltk. Работают они вот так:

In [48]:
from nltk.stem.snowball import SnowballStemmer

In [49]:
stemmer = SnowballStemmer('russian')

In [52]:
[(word, stemmer.stem(word)) for word in text.split()][:15]

[('Сегодняшняя', 'сегодняшн'),
 ('заметка', 'заметк'),
 ('будет', 'будет'),
 ('о', 'о'),
 ('сервисе', 'сервис'),
 ('отслеживания', 'отслеживан'),
 ('активности', 'активн'),
 ('пользователя', 'пользовател'),
 ('—', '—'),
 ('CrazyEgg', 'CrazyEgg'),
 ('.', '.'),
 ('Я', 'я'),
 ('не', 'не'),
 ('знаю', 'зна'),
 ('кому', 'ком')]

In [95]:
stemmer.stem("пролить")

'прол'

In [99]:
stemmer.stem("прольёт")

'прольет'

In [94]:
stemmer.stem("пролом")

'прол'

Недостатки стемминга достаточно очевидные:  
1) с супплетивными формами или редкими окончаниями слова стемминг работать не умеет  
2) к одной могут приводится разные слова  
3) редкие окончания не отбрасываются

# Лемматизация

Лемматизация - это замена словоформы слова в парадигме на какую-то заранее выбранную стадартную форму (лемму)*. 



Например, для разных форм глагола леммой будет неопределенная форма, а для существительного форма мужского рода единственного числа. Это позволяет избавить от недостатков стемминга (будет, был - одна лемма), (пролить, пролом - разные). Однако лемматизация значительно сложнее. 

\* - (Значения слов "слово", "слоформа", "парадигма" приблизительно соответствует тому, которое использует Зализняк вот тут - http://inslav.ru/images/stories/pdf/2002_Zalizniak_RIS_i_statji.pdf (стр. 21-22))

К счастью есть готовые хорошие лемматизаторы. Для русского основых варианта два: Mystem и Pymorphy.


### Mystem


Майстем работает немного лучше и сам токенизирует,
поэтому можно в него засовывать сырой текст.

In [159]:
# mystem.lemmatize функция лемматизации в майстеме
# сам объект mystem нужно заранее инициализировать
# мы сделали это в начале тетрадки строчкой "mystem = Mystem()"
mystem.lemmatize(text)[:10]

['\n', 'сегодняшний', ' ', 'заметка', ' ', 'быть', ' ', 'о', ' ', 'сервис']

In [167]:
# Если нужна грамматическая информация или надо сохранить ненормализованный текст,
# есть функция mystem.analyze
words_analized = mystem.analyze(t)

In [166]:
t = 'Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.'

In [168]:
# возвращает она список словарей
# каждый словарь имеет либо одно поле 'text' (когда попался пробел) или text и analysis
# в analysis снова список словарей с вариантами разбора (первый самый вероятный)
# поля в analysis - 'gr' - грамматическая информация, 'lex' - лемма
# analysis - может быть пустым списком
words_analized[:100]

[{'analysis': [{'lex': 'сегодняшний', 'wt': 1, 'gr': 'A=им,ед,полн,жен'}],
  'text': 'Сегодняшняя'},
 {'text': ' '},
 {'analysis': [{'lex': 'заметка', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}],
  'text': 'заметка'},
 {'text': ' '},
 {'analysis': [{'lex': 'быть',
    'wt': 0.1492564349,
    'gr': 'V,нп=непрош,ед,изъяв,3-л'}],
  'text': 'будет'},
 {'text': ' '},
 {'analysis': [{'lex': 'о', 'wt': 0.9737875835, 'gr': 'PR='}], 'text': 'о'},
 {'text': ' '},
 {'analysis': [{'lex': 'сервис', 'wt': 1, 'gr': 'S,муж,неод=пр,ед'}],
  'text': 'сервисе'},
 {'text': ' '},
 {'analysis': [{'lex': 'отслеживание',
    'wt': 1,
    'gr': 'S,сред,неод=(вин,мн|род,ед|им,мн)'}],
  'text': 'отслеживания'},
 {'text': ' '},
 {'analysis': [{'lex': 'активность',
    'wt': 1,
    'gr': 'S,жен,неод=(пр,ед|вин,мн|дат,ед|род,ед|им,мн)'}],
  'text': 'активности'},
 {'text': ' '},
 {'analysis': [{'lex': 'пользователь',
    'wt': 1,
    'gr': 'S,муж,од=(вин,ед|род,ед)'}],
  'text': 'пользователя'},
 {'text': ' — '},
 {'analys

In [1153]:
print('Слово - ', words_analized[0]['text'])
print('Разбор слова - ', words_analized[0]['analysis'][0])
print('Лемма слова - ', words_analized[0]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', words_analized[0]['analysis'][0]['gr'])

Слово -  Сегодняшняя
Разбор слова -  {'lex': 'сегодняшний', 'gr': 'A=им,ед,полн,жен'}
Лемма слова -  сегодняшний
Грамматическая информация слова -  A=им,ед,полн,жен


In [1137]:
#леммы можно достать в одну строчку
[parse['analysis'][0]['lex'] for parse in words_analized if parse.get('analysis')][:10]

['сегодняшний',
 'заметка',
 'быть',
 'о',
 'сервис',
 'отслеживание',
 'активность',
 'пользователь',
 'я',
 'не']

Mystem умеет разбивать текст на предложения, но через питоновский интерфейс это сделать не получится. Нужно скачать mystem отсюда - https://yandex.ru/dev/mystem/

После этого сохранить текст в файл.

In [175]:
f = open('text.txt', 'w')
f.write(text)
f.close()

Из командной строки или из питона запустить майстем на нашем файле

In [ ]:
# про параметры почитайте в ./mystem -h
os.system(' ./mystem -iscd --format json text.txt text_parsed.txt')

В целевом файле теперь лежит разобранный текст в jsonlines (json на каждой строчке)

In [183]:
t = [json.loads(line) for line in open('text_parsed.txt')]

По индексам в этом списке лежат предложения.

Ещё так вызывать майстем может понадобиться, если важна скорость.

Недостатки Mystem: это продукт Яндекса с некоторыми ограничениями на использование, больше он не развивается.

Важным достоинством Mystem является то, что он работает не с отдельными словами, а с целым предложением. При определении нужной леммы учитывается контекст, что позволяет во многих случаях разрешать омонимию.

### Pymorphy

Pymorphy - открытый и развивается (можно поучаствовать на гитхабе)

Ссылка на репозиторий: https://github.com/kmike/pymorphy2

Попробуйте сразу установить быструю версию (pip install pymorphy2[fast])

У него нет втстроенной токенизации и он расценивает всё как слово. Когда есть несколько вариантов, он выдает их с вероятностостями, которые расчитатны на корпусе со снятой неоднозначностью. Это лучше стемминга, но хуже майстема.

In [108]:
# основная функция - pymorphy.parse
words_analized = [morph.parse(token) for token in word_tokenize(clean_habr_texts[2])]

In [118]:
# Она похожа на analyze в майстеме только возрващает список объектов Parse
# Первый в списке - самый вероятный разбор (у каждого есть score)
# Информация достается через атрибут (Parse.word - например)
# Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать
# части речи или другие категории
print('Первое слово - ', words_analized[0][0].word)
print('Разбор первого слова - ', words_analized[0][0])
print('Лемма первого слова - ', words_analized[0][0].normal_form)
print('Грамматическая информация первого слова - ', words_analized[0][0].tag)
print('Часть речи первого слова - ', words_analized[0][0].tag.POS)
print('Род первого слова - ', words_analized[0][0].tag.gender)
print('Число первого слова - ', words_analized[0][0].tag.number)
print('Падеж первого слова - ', words_analized[0][0].tag.case)

Первое слово -  сегодняшняя
Разбор первого слова -  Parse(word='сегодняшняя', tag=OpencorporaTag('ADJF femn,sing,nomn'), normal_form='сегодняшний', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'сегодняшняя', 382, 7),))
Лемма первого слова -  сегодняшний
Грамматическая информация первого слова -  ADJF femn,sing,nomn
Часть речи первого слова -  ADJF
Род первого слова -  femn
Число первого слова -  sing
Падеж первого слова -  nomn


## Дополнительная очистка текста

Пунктуация часто совсем не нужна и поэтому можно выбросить её заранее. Если нужно обрабатывать много текста, это может немного ускорить процесс.


In [1154]:
# Оставим только буквено-численные токены
# Это не самый лучший вариант, так как удалятся сокращения с точкой, слова через дефис
text = 'В этом случае слова вроде т.к. и по-другому не пройдут фильтр и будут удалены.'
good_tokens = [word for word in word_tokenize(text) if word.isalnum()]
[morph.parse(token)[0].normal_form for token in good_tokens]

['в',
 'это',
 'случай',
 'слово',
 'вроде',
 'и',
 'не',
 'пройти',
 'фильтр',
 'и',
 'быть',
 'удалить']

In [198]:
#можно сделать фильтр по длине
good_tokens = [word.strip(string.punctuation) for word in word_tokenize(text) if len(word) > 2 and len(word) < 35]
[morph.parse(token)[0].normal_form for token in good_tokens][:10]

['сегодняшний',
 'заметка',
 'быть',
 'сервис',
 'отслеживание',
 'активность',
 'пользователь',
 'crazyegg',
 'знать',
 'кома']

Можно убрать стоп-слова (предлоги, союзы, местоимения, частотные слова). Сам термин стоп-слово происходит из информационного поиска, первый раз его упомянул [Питер Лун](https://en.wikipedia.org/wiki/Hans_Peter_Luhn) в 1959.  
Удаление таких слов позволяло сократить размер индекса и не сильно испортить выдачу или даже улучшить её, поднимая релевантность документам со значимыми словами. Со временем от такой практики, в основном, отказались - память стала дешевой (и повились всякие алгоритмы для сокращения потребления памяти), а для учёта значимости придумали IDF.  

Во многих поисковых движках стоп-слова всё ещё используются. Часто их используют и в практических задачах (классификации, тематическом моделировании). 

In [120]:
# стоп-слова есть в nltk
stops = stopwords.words('russian')
print(stops)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

Список не идеальный и его можно расширять под свои задачи.

In [144]:
words_normalized = [morph.parse(token)[0].normal_form for token in good_tokens]
[word for word in words_normalized if word not in stops]

['это',
 'случай',
 'слово',
 'вроде',
 'т.к',
 'по-другому',
 'пройти',
 'фильтр',
 'удалить']

# Универсальная предобработка

Всё, что мы рассмотрели выше, так или иначе привязано к особенностям конкретного языка (токенизацию мы делали по пробелам, а они есть не везде; стемминг и лемматизация вообще работают только для целевого языка). Есть и универсальные методы предобработки. 

Например, **byte-pair-encoding**. Этот, изначально придуманный для кодировки алгоритм, сейчас используется почти во всех state-of-the-art моделях машинного перевода.  

При этом идея там не такая сложная. Изначально текст рассматривается как последовательность символов. На каждом шаге алгоритма в один токен склеивается самое частотное сочетание символов. Единственным параметром алгоритма является количество таких шагов. Чем больше шагов, тем конткретнее становятся токены (сначала собираются частотные морфемы, корни, потом слова, потом уже словосочетания). Если алгоритм не остановить, то весь текст соберется в один токен. 

Статья, в которой предлагается его использовать для машинного перевода - https://www.aclweb.org/anthology/P16-1162

Минимальная реализация совсем короткая:

In [115]:
import re, collections
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

# vocab = {'l o w </w>' : 5, 'l o w e r </w>' : 2,
# 'n e w e s t </w>':6, 'w i d e s t </w>':3}
num_merges = 300

for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
print(vocab)

{'се го д ня ш ня я</w>': 1, 'за мет ка</w>': 1, 'будет</w>': 3, 'о</w>': 5, 'сервисе</w>': 2, 'отслеживания</w>': 5, 'активности</w>': 2, 'пользовате ля</w>': 1, '—</w>': 18, 'crazyegg</w>': 3, '.</w>': 54, 'я</w>': 6, 'не</w>': 12, 'зна ю</w>': 1, 'ко му</w>': 1, 'обяза н</w>': 1, 'сервис</w>': 14, 'так им</w>': 1, 'го во ря щ им</w>': 1, 'име не м</w>': 1, ',</w>': 53, 'но</w>': 8, 'о но</w>': 1, 'работает</w>': 3, 'и</w>': 21, 'хорошо</w>': 2, 'за помина ется</w>': 1, '?</w>': 6, 'отли чно</w>': 1, '!</w>': 2, 'что</w>': 3, 'это</w>': 7, 'та ко е</w>': 1, 'как</w>': 4, 'уже</w>': 2, 'с ка за но</w>': 1, 'вы ше</w>': 1, 'для</w>': 17, 'пе ре ме ще ния</w>': 1, 'пользователей</w>': 3, 'по</w>': 2, 'сайт у</w>': 1, 'к то</w>': 1, 'к уд а</w>': 1, 'к ли к ну л</w>': 1, 'каки е</w>': 1, 'ссы л ки</w>': 1, 'наиболее</w>': 3, 'популя р ные</w>': 1, 'то му</w>': 1, 'под об ная</w>': 1, 'разно ро д ная</w>': 1, 'ин форма ци я</w>': 1, 'юзабилистов</w>': 2, 'по з во ля ет</w>': 1, 'отслежива

Разумеется, это не самая эффективная реализация. Но об этом уже позаботились умные люди и вы можете воспользоваться готовыми решениями.

Совсем недавно ребята из ВК сделали библиотеку YouTokenToMe, по бенчмаркам оно обходит другие популярные реализации (SentencePiece от Гугла, например). Пост на медиуме - https://medium.com/@vktech/youtokentome-a-tool-for-quick-text-tokenization-from-the-vk-team-aa6341215c5a

По какой-то причине в jupyter не работает обучение модели. Обучить модель можно в командной строке.

In [ ]:
# yttm bpe --data text.txt --model model --vocab_size 1000

In [1]:
import random
import youtokentome as yttm

Модель загружается вот так.

In [5]:
bpe = yttm.BPE(model='model')

Преобразовать текст можно так.

In [8]:
# Two types of tokenization
print(bpe.encode([text], output_type=yttm.OutputType.SUBWORD))

[['▁Сегодняшняя', '▁заметка', '▁будет', '▁о', '▁сервисе', '▁отслеживания', '▁активности', '▁пользователя', '▁—', '▁', '<', 'a', '▁', 'h', 'r', 'e', 'f', '="h', 't', 't', 'p', ':', '//www', '.', 'c', 'r', 'a', 'z', 'y', 'e', 'g', 'g', '.', 'c', 'o', 'm', '"', '▁', 't', 'i', 't', 'l', 'e', '="', 'С', 'у', 'ма', 'с', 'шед', 'ши', 'е', '▁я', 'й', 'ц', 'а', '">', 'CrazyEgg', '</', 'a', '>', '.', '▁Я', '▁не', '▁знаю', '▁ко', 'му', '▁обязан', '▁сервис', '▁таки', 'м', '▁г', 'ов', 'ор', 'я', 'щ', 'им', '▁и', 'ме', 'не', 'м', ',', '▁но', '▁о', 'но', '▁работает', ',', '▁и', '▁хорошо', '.', '▁Запоминается?', '▁Отлично!', '<b', 'r', '><b', 'r', '><', 'i', 'm', 'g', '▁', 's', 'r', 'c', '="h', 't', 't', 'p', ':', '//', 'i', 'm', 'g', '1', '7', '2', '.', 'i', 'm', 'a', 'g', 'e', 's', 'h', 'a', 'c', 'k', '.', 'u', 's', '/', 'i', 'm', 'g', '1', '7', '2', '/8', '4', '3', '4', '/', '1', '8', '2', '7', '4', '6', '5', '8k', 'c', '4', '.', 'p', 'n', 'g', '"', '▁', 'a', 'l', 't', '="', 'С', 'умасшедшее', '▁я'

Выглядит это все как какие-то бессмысленные обрезки текста. Но на самом деле это очень эффективный способ токенизации. Когда дойдем до классификаторов и нейронных сетей, вы всё увидите сами.